Netflix wants to identify similar movies based on movie characteristics.

In [111]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from wordcloud import WordCloud

# Display preference
warnings.filterwarnings('ignore')
pd.set_option('Display.max_columns', 100)
pd.set_option('Display.max_rows', 10000)

In [112]:
netflix = pd.read_csv('../../data/netflix_titles.csv')

In [113]:
netflix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6234 entries, 0 to 6233
Data columns (total 12 columns):
show_id         6234 non-null int64
type            6234 non-null object
title           6234 non-null object
director        4265 non-null object
cast            5664 non-null object
country         5758 non-null object
date_added      6223 non-null object
release_year    6234 non-null int64
rating          6224 non-null object
duration        6234 non-null object
listed_in       6234 non-null object
description     6234 non-null object
dtypes: int64(2), object(10)
memory usage: 584.6+ KB


In [114]:
genre_length = 0

for genre in netflix['listed_in']:
    genre = genre.split(',')

    if len(genre) > genre_length:
        genre_length = len(genre)
        
print(genre_length)    

3


In [118]:
def genre(listed_in):
    genre_list = listed_in.split(',')
    
    if len(genre_list) == 3:
        return genre_list[0], genre_list[1], genre_list[2]
    elif len(genre_list) == 2:
        return genre_list[0], genre_list[1], np.nan
    else:
        return genre_list[0], np.nan, np.nan

In [119]:
netflix['genre0'], netflix['genre1'], netflix['genre2'] = zip(*netflix['listed_in'].apply(genre))

In [122]:
netflix[['genre0', 'genre1', 'genre2']].head()

,genre0,genre1,genre2
0,Children & Family Movies,Comedies,NaN
1,Stand-Up Comedy,NaN,NaN
2,Kids' TV,NaN,NaN
3,Kids' TV,NaN,NaN
4,Comedies,NaN,NaN
